In [ ]:
from __future__ import print_function
import cv2
import numpy as np
from numpy import pi, exp, sqrt
import pickle
from matplotlib import pyplot as plt
from IPython import display
import time
#import pylab as pl

%matplotlib inline 
#notebook or inline

###https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2, matchfilename = 'matches.jpg'):
    # Convert images to grayscale
    #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    im1Gray = im1
    im2Gray = im2

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite(matchfilename, imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
   
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width = im2.shape
    #height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h



def alignImages_light(im1, im2):

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
   
    # Find homography
    #h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    #height, width = im2.shape
    #height, width, channels = im2.shape
    #im1Reg = cv2.warpPerspective(im1, h, (width, height))
    
    
    # Find 
    # Find homography
    h, mask = cv2.estimateAffinePartial2D(points1, points2, cv2.RANSAC)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpAffine(im1, h, (width, height))

    
    return im1Reg, h

def normalize(x):
    x = (x - np.min(x))/(np.max(x)-np.min(x))
    #x = (x - np.percentile(x,10))/(np.percentile(x,90)-np.percentile(x,10))
    return x

def twodgaussian(s,r):
    #  generate a (2k+1)x(2k+1) gaussian kernel with mean=0 and sigma = s
    probs = [exp(-z*z/(2*s*s))/sqrt(2*pi*s*s) for z in range(-r,r+1)] 
    kernel = np.outer(probs, probs)
    return kernel

In [ ]:
import os
# make folder to contain connected files
#os.makedirs('..\\connected',exist_ok=True)

downscaling = 1
Radius = 35/2;#35/2; #mm
Res = 600*downscaling; #PSI
Scale = 100; # per cent
ConversionFactor = 0.039370078740157*Res*Scale/100; # mm to pix
Radius_in_pix = np.ceil(Radius * ConversionFactor); 

In [ ]:
import glob

files = []

for file in glob.glob("./091919_scaling/*.tif"):
    files.append(file)
files = np.array(files)

In [ ]:
num_t = 240
num_s = 1
num_ch = 3
file_matrix = np.reshape(files[0:num_t*num_s*num_ch],[num_t,num_s,num_ch])

In [ ]:
def starting_pos(pos_ind):
    i , j = pos_ind
    y = np.ceil(shape_field[0]*0.9*i).astype(int)
    x = np.ceil(shape_field[0]*0.9*j).astype(int)
    return y, x

In [ ]:
file_matrix

In [ ]:
current_image = cv2.imread(file_matrix[0,0,0],cv2.IMREAD_GRAYSCALE)
shape_field = np.shape(current_image)
#canvas = np.zeros(np.ceil(np.dot(shape_field,(1.0+0.9*2))).astype(int))
#canvas = np.zeros((shape_field[0],np.ceil((shape_field[1]*(1.0+0.9))).astype(int)))
#canvas = np.zeros(np.ceil(np.dot(shape_field,(1.0+0.9*1))).astype(int))
canvas = np.zeros(np.ceil(np.dot(shape_field,(1.0))).astype(int))


#pos_inds = [(i,j) for i in range(3) for j in range(3)]
#pos_inds = [[(i,j) for j in range(3)] if i%2==0 else [(i,j) for j in reversed((range(3)))] for i in range(3)]
#pos_inds = [j for i in pos_inds for j in i]
#pos_inds = [(0,0),(0,1),(0,2),(1,2),(1,1),(1,0),(2,0),(2,1),(2,2)]
#pos_inds = [[(i,j) for j in range(2)] if i%2==0 else [(i,j) for j in reversed((range(2)))] for i in range(2)]
#pos_inds = [j for i in pos_inds for j in i]
pos_inds = [[(i,j) for j in range(1)] if i%2==0 else [(i,j) for j in reversed((range(1)))] for i in range(1)]
pos_inds = [j for i in pos_inds for j in i]

field_h , field_w = shape_field

ch = 2
imRegs = np.zeros((num_t,num_ch,np.shape(canvas)[0],np.shape(canvas)[1]))

for t in range(num_t):
    for s in range(num_s):
        for ch in range(num_ch):
            file = file_matrix[t,s,ch]
            current_image = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
            y , x = starting_pos(pos_inds[s])
            canvas[y:y+field_h,x:x+field_w] = current_image
            imRegs[t,ch] = canvas

In [ ]:
%matplotlib inline
for i in range(len(imRegs)):
    f = plt.figure(figsize = [15,15])
    plt.imshow(imRegs[i,2])
    plt.show()
    display.clear_output(wait=True)

In [ ]:
%matplotlib inline
import pylab as pl
from IPython import display
for i in range(10):
    pl.clf()
    display.display(pl.gcf())

### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use
from scipy import ndimage
import matplotlib.animation as animation

smooth_filter = twodgaussian(1.5,5)

roberts_cross_v = np.array( [[ 1, 0],
                             [ 0, -1]])

roberts_cross_h = np.array( [[ 0, 1],
                             [ -1, 0]])


plots = []
edge2 = None
for i in range(len(imRegs)-1):

    fig = plt.figure(figsize = [15,15])
    if edge2 is not None:
        edge1 = edge2
    else:
        smoothed = ndimage.convolve( imRegs[i,2], smooth_filter )
        vertical = ndimage.convolve( smoothed, roberts_cross_v )
        horizontal = ndimage.convolve( smoothed, roberts_cross_h )
        edge1 = np.sqrt( np.square(horizontal) + np.square(vertical))

    smoothed = ndimage.convolve( imRegs[i+1,2], smooth_filter )
    vertical = ndimage.convolve( smoothed, roberts_cross_v )
    horizontal = ndimage.convolve( smoothed, roberts_cross_h )
    edge2 = np.sqrt( np.square(horizontal) + np.square(vertical))
    
    normalized = (normalize(edge1*2-edge2*2)*255).astype(int)
    plot =plt.imshow(ndimage.convolve(normalized,smooth_filter))
    plt.show()
    display.clear_output(wait=True)
    time.sleep(0.001)

In [ ]:
%matplotlib inline

### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use
from scipy import ndimage
import matplotlib.animation as animation

smooth_filter = twodgaussian(1.5,5)

roberts_cross_v = np.array( [[ 1, 0],
                             [ 0, -1]])

roberts_cross_h = np.array( [[ 0, 1],
                             [ -1, 0]])




plots = []

#for i in range(5):
for i in range(0,len(imRegs)-1,1):
    fig = plt.figure(figsize = [15,15])
    smoothed = ndimage.convolve( imRegs[i], smooth_filter )
    vertical = ndimage.convolve( smoothed, roberts_cross_v )
    horizontal = ndimage.convolve( smoothed, roberts_cross_h )
    edge = np.sqrt( np.square(horizontal) + np.square(vertical))
    
    normalized = (normalize(edge)*255).astype(int)
    plot =plt.imshow(ndimage.convolve(normalized,smooth_filter))
    plots.append([plot])
    plt.show()
    #display.clear_output(wait=True)

In [ ]:
%matplotlib inline

### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use
from scipy import ndimage
import matplotlib.animation as animation

smooth_filter = twodgaussian(2,5)
smooth_filter_2 = twodgaussian(1.5,5)

roberts_cross_v = np.array( [[ 1, 0],
                             [ 0, -1]])

roberts_cross_h = np.array( [[ 0, 1],
                             [ -1, 0]])


plots = []
stack = []

#for i in range(10):
for i in range(0,len(imRegs),1):
    fig = plt.figure(figsize = [15,15])
    smoothed = ndimage.convolve( normalize(imRegs[i,2]),smooth_filter)
    vertical = ndimage.convolve( smoothed, roberts_cross_v )
    horizontal = ndimage.convolve( smoothed, roberts_cross_h )
    edge = np.sqrt( np.square(horizontal) + np.square(vertical))
    
    normalized = (normalize(edge)*255).astype(int)
    plot =plt.imshow(ndimage.convolve(normalized,smooth_filter_2))
    stack.append(normalized)
    plt.text(100,200,str(i),backgroundcolor = 'w',color = 'b')
    #plot = plt.imshow(imRegs[i])
    plots.append([plot])
    plt.show()
    display.clear_output(wait=True)

In [ ]:
len(stack)

In [ ]:
cv2.isContourConvex(centroids[0])

In [ ]:
sorted = np.sort([x for x in stats[:,4] if x > 1000])

In [ ]:
np.mean(sorted[0:3])*1.5

In [ ]:
len((centroids[:,0]>50))

In [ ]:
np.shape(imRegs[0,0])

In [ ]:
plt.figure(figsize = [15,15])
plt.imshow(imRegs[0,0])

In [ ]:
frame = 0
img = imRegs[frame,2]

In [ ]:
def extract_stats(gray,frame):
    from scipy import ndimage
    from skimage.morphology import watershed
    gray = gray

    cellsummary = np.empty((0,6), 'int') #picture, GFP, RFP, area, x , y

    ### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use


    smooth_filter = twodgaussian(2,5)
    smooth_filter_2 = twodgaussian(1.5,5)

    roberts_cross_v = np.array( [[ 1, 0],
                                 [ 0, -1]])

    roberts_cross_h = np.array( [[ 0, 1],
                                 [ -1, 0]])

    smoothed = ndimage.convolve( normalize(gray),smooth_filter)  #imRegs[0,2]
    vertical = ndimage.convolve( smoothed, roberts_cross_v )
    horizontal = ndimage.convolve( smoothed, roberts_cross_h )
    edge = np.sqrt( np.square(horizontal) + np.square(vertical))

    normalized = (normalize(edge)*255).astype('uint8')

    ret, thresh = cv2.threshold(normalized,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # noise removal
    kernel = np.ones((2,2),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

    # Marker labelling
    connectivity = 4  # You need to choose 4 or 8 for connectivity type  https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connected-components-with-stats-in-python
    num_markers, markers, stats, centroids = cv2.connectedComponentsWithStats(opening , connectivity , cv2.CV_16U+cv2.CC_STAT_AREA )

    # set lower limit for single cell
    threshold_single_l = 600
    # set upper limit for single cell
    sorted = np.sort([x for x in stats[:,4] if x > threshold_single_l])
    #if len(sorted) > 10:
    #    threshold_single = np.median(sorted[0:10])*2.5
    #else:
    #    threshold_single = 15000
    threshold_single = 150000

    # Choose markers of area within a certain range
    chosen = np.squeeze(np.where((stats[:,4]>threshold_single_l)* (stats[:,4]<threshold_single)  # something good to read https://stackoverflow.com/questions/33747908/output-of-numpy-wherecondition-is-not-an-array-but-a-tuple-of-arrays-why
                       *(centroids[:,0]>50)*(centroids[:,0]<field_w-50)
                       *(centroids[:,1]>50)*(centroids[:,1]<field_h-50)) )

    if chosen.shape is not ():
        plt.figure(figsize = [15,15])
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1

        #plt.imshow(markers.astype(np.uint8))
        plt.imshow(gray)
        #plt.imshow(imRegs[frame,0])
        for int, c in enumerate(centroids[chosen]):
            plt.text(c[0], c[1], str(int)+'_'+str(stats[chosen[int],4]),fontsize = 20,color = 'r')
       
      
        centroids_current = centroids[chosen]
        [plt.plot(x[0],x[1],'r.') for x in centroids_current]

        plt.show()
        

    nummask = np.zeros(np.shape(markers))
    for index, cell in enumerate(chosen+1):
        nummask[markers == cell] = index
    nummask = nummask.astype('uint8')

    # Now, mark the region of unknown with zero

    watershed = watershed(gray,nummask)
    #img[watershed == -1] = [255,0,0]



    # calculate the sum of intensity in other channels
    record = watershed*0  
    current_frame_GFP = np.zeros(len(chosen))
    current_frame_RFP = np.zeros(len(chosen))
    current_frame_x = np.zeros(len(chosen))
    current_frame_y = np.zeros(len(chosen))
    for cell_ind, cell in enumerate(set([i for i in watershed.flatten()])):
        singlecellmask = np.squeeze(np.array([watershed == cell+1]).astype('int'))
        w_area = np.sum(singlecellmask) # watershed_area
        centroid_y,centroid_x = ndimage.measurements.center_of_mass(singlecellmask)
        
        if (w_area < threshold_single*1.5)*(w_area>0):
            singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask).astype('int')
            current_frame_GFP[cell_ind] = singlecellGFP #/w_area
            singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask).astype('int')
            current_frame_RFP[cell_ind] = singlecellRFP #/w_area
            current_frame_x[cell_ind],current_frame_y[cell_ind] = centroid_x, centroid_y
            summary_current_cell = [frame,singlecellGFP,singlecellRFP, w_area, centroid_x, centroid_y]
            print(summary_current_cell)
            cellsummary = np.append(cellsummary,[summary_current_cell],axis = 0)
            record += watershed*(watershed == cell+1)
    w_chosen = chosen[list(set(record.flatten()))] #watershed chosen
    
    print(set(record.flatten()))
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,0]*(record>0))
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(current_frame_x[int], current_frame_y[int], 'r.')
        plt.text(current_frame_x[int], current_frame_y[int], str(current_frame_GFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,1]*(record>0))
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(current_frame_x[int], current_frame_y[int], 'r.')
        plt.text(current_frame_x[int], current_frame_y[int], str(current_frame_RFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(watershed)
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(current_frame_x[int], current_frame_y[int], 'r.')
        #plt.text(current_frame_x[int], current_frame_y[int], str(current_frame_GFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,1]*(record>0))
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(current_frame_x[int], current_frame_y[int], 'r.')
    plt.show()

    return cellsummary

In [ ]:
extract_stats(img,frame)

In [ ]:
Two = [i for i in range(33)]
Four = [i for i in range(33,33+33)]
Five = [i for i in range(65,65+33)]
Six = [i for i in range(98,98+34)]
Seven = [i for i in range(132,132+35)]
Eight = [i for i in range(167,167+35)]
Nine = [i for i in range(202,202+37)]
strainlist = [Two,Four,Five,Six,Seven,Eight,Nine]
strainlist_n = ['Two','Four','Five','Six','Seven','Eight','Nine']

In [ ]:
for i in range(240):
    print(str(file_matrix[i])+str(np.array([str(index)+'_'+str(j) for index, i in enumerate(strainlist) for j in i]).flatten()[i]))

In [ ]:
os.getcwd()

In [ ]:
#frames = range(len(stack))
for strain_n,strain in zip(strainlist_n,strainlist):
    cellsummary = np.empty((0,6), 'int')
    for frame in strain:
        #print('frame'+str(frame))
        img = imRegs[frame,2]
        cellsummary_current = extract_stats(img,frame)
        cellsummary = np.append(cellsummary,cellsummary_current,axis=0)
    print('./092319_cellsummary_'+str(strain_n)+'.npy')
    outfile = open('./092319_cellsummary_'+str(strain_n)+'.npy','wb')
    np.save(outfile, cellsummary)

In [ ]:
cellsummary = np.empty((0,6), 'int')
for strain_n,strain in zip(strainlist_n,strainlist):
    outfile = open('./092319_cellsummary_'+str(strain_n)+'.npy','rb')
    cellsummary = np.append(cellsummary,np.load(outfile),axis=0)

In [ ]:
np.where(cellsummary[:,0] == cellsummary[0,0])

In [ ]:
cellsummary_trimmed = np.empty((0,6),'int')
for item in cellsummary:
    print(item)
    print(cellsummary_trimmed)
    if item[area] < 2*np.median(cellsummary[np.where(cellsummary[:,0]==item[0]),area]) and \
    item[area]>0.75*np.median(cellsummary[np.where(cellsummary[:,0]==item[0]),area]):
        cellsummary_trimmed = np.append(cellsummary_trimmed,[item],axis = 0)

In [ ]:
np.shape(cellsummary)

In [ ]:
    plt.figure(figsize = [20,10])
    x = cellsummary_trimmed[:,0]
    #y = cellsummary[strain,GFP]/cellsummary[strain,area]**(3/2)
    y = cellsummary_trimmed[:,area]#/cellsummary_trimmed[:,area]**(3/2)
    #y = cellsummary[strain,GFP]/cellsummary[strain,RFP]
    
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    line = slope*x+intercept
    colormap = plt.cm.jet #nipy_spectral, Set1,Paired   
    colors = [colormap(i) for i in np.linspace(0, 1,len(x))]

    for i in range(len(x)):
        plt.plot(x[i],y[i],'*',color = colors[i], alpha=0.3)
    #plt.plot( x, line)#,'r',label = "")
    for separator in [33,65,98,132,167,202]:
        plt.axvline(x=[separator],dashes=[10, 5, 10, 5],color = 'black')

    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    #x=0
    #y=0
    #x = cellsummary[WHI3,RFP]
    #y = cellsummary[WHI3,GFP]/cellsummary[WHI3,area]**(3/2)
    #slope, intercept, r_value, p_value, std_err = linregress(x,y)
    #line = slope*x+intercept
    #whi3 = plt.plot(x,y,'g*')
    #plt.plot(x , line,'g',label = "WHI3")

    print(slope)
    print(r_value)
    #plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
    #plt.xlim(0,5) #area
    #plt.xlim(0,20000)  # RFP
    
    #plt.xlim(0,5000000)
    #plt.ylim(0,50000)
    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    plt.xlabel("sample",size = 15)
    plt.ylabel("area", size = 15)
    plt.legend(fontsize = 15)

In [ ]:
len(cellsummary_trimmed)

In [ ]:
x_all = cellsummary_trimmed[:,0]
y_all = cellsummary_trimmed[:,area]

In [ ]:
#frame,singlecellGFP,singlecellRFP, w_area, centroid_x, centroid_y
x_all = cellsummary_trimmed[:,area]**(3/2)
x_all = cellsummary_trimmed[:,0]
#y = cellsummary[strain,GFP]/cellsummary[strain,area]**(3/2)
y_all = cellsummary_trimmed[:,GFP]/cellsummary_trimmed[:,area]**(3/2)
#y = cellsummary[strain,GFP]/cellsummary[strain,RFP]

In [ ]:
import pickle
with open('record','rb') as f:
    records = pickle.load(f)
    
colormap = plt.cm.jet #nipy_spectral, Set1,Paired   
colors = [colormap(i) for i in np.linspace(0, 1,len(x))]
plt.figure(figsize = [8,5])
separator =[0]
color = ['r','g','y']
color = ['r','y']
marker = ['r*','g*','y*']
marker = ['r*','y*']
strain_list = ['Cln3','Whi3','Hta2']  
strain_list = ['Cln3','Hta2']
for ind,record in enumerate([records[0],records[2]]):
    x = record[0]
    x = x - np.min(x) + separator[len(separator)-1]
    y = record[1]
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    line = slope*x+intercept
    plt.plot(x,y,marker[ind])
    plt.plot(x , line,color[ind],label = strain_list[ind])
    separator.append(np.max(x))
    print(slope)
    print(r_value)
    
WHI5 = range(133,167)
ADE1 = range(167,203)
CON = range(203,len(x_all))
strain_list = ["Whi5","Ade1","No GFP"]
color = ['g','b','black']
marker = ['g*','b*','k*']
for ind,strain in enumerate([WHI5,ADE1,CON]):
    #plt.figure(figsize = [5,5])
    strain_range = [i for i in range(len(cellsummary_trimmed)) if cellsummary_trimmed[i,0] in strain]
    x = x_all[strain_range]
    x = x-np.min(x)+ separator[len(separator)-1]
    y = y_all[strain_range]
    separator.append(np.max(x))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    line = slope*x+intercept
    plt.plot(x,y,marker[ind])
    plt.plot(x , line,color[ind],label = strain_list[ind])
    
    print(slope)
    print(r_value)
    



    
    #plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
    #plt.xlim(0,12)
    #[print(str([x,y]) + ';') for x, y in zip(x,y)]
    #plt.ylim(0,1)

    plt.xlabel("Sample",size = 15)
    plt.ylabel("FP/volume", size = 15)
    plt.legend(fontsize = 15)
    #plt.show()
plt.savefig('FPvolume_vs_sample.pdf',bbox_inches='tight',pad_inches=0.2)

In [ ]:
len(cellsummary_trimmed[:,0])

In [ ]:
from scipy.stats import linregress
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
two = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(33)]
four = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(33,33+33)]
five = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(65,65+33)]
six = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(98,98+34)]
seven = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(132,132+35)]
eight = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(167,167+35)]
nine = [i for i, ind in enumerate(cellsummary_trimmed[:,0]) if ind in range(202,202+37)]
smallstrainlist = [two,four,five,six,seven,eight,nine]
for strain_n,strain in zip(strainlist_n,smallstrainlist):
    plt.figure(figsize = [5,5])
    x = cellsummary_trimmed[strain,area]**(3/2)
    y = cellsummary_trimmed[strain,RFP]
    x = cellsummary_trimmed[strain,RFP]
    y = cellsummary_trimmed[strain,GFP]/cellsummary_trimmed[strain,area]**(3/2)

    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    line = slope*x+intercept
    colormap = plt.cm.jet #nipy_spectral, Set1,Paired   
    colors = [colormap(i) for i in np.linspace(0, 1,len(strain))]

    for i in range(len(x)):
        plt.plot(x[i],y[i],'*',color = colors[i], alpha=0.3)
    plt.plot( x, line)#,'r',label = "")


    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    #x=0
    #y=0
    #x = cellsummary[WHI3,RFP]
    #y = cellsummary[WHI3,GFP]/cellsummary[WHI3,area]**(3/2)
    #slope, intercept, r_value, p_value, std_err = linregress(x,y)
    #line = slope*x+intercept
    #whi3 = plt.plot(x,y,'g*')
    #plt.plot(x , line,'g',label = "WHI3")

    print(slope)
    print(r_value)
    #plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
    #plt.xlim(0,5) #area
    #plt.xlim(0,20000)  # RFP
    
    #plt.xlim(0,5000000)
    #plt.ylim(0,50000)
    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    plt.xlabel("RFP",size = 15)
    plt.ylabel("GFP/volume", size = 15)
    plt.legend(fontsize = 15)


In [ ]:
for i in range(len(x)):
    plt.plot(i,i,'*',color=colors[i])

In [ ]:
def extract_stats(gray,frame):
    from scipy import ndimage
    from skimage.morphology import watershed
    gray = gray

    cellsummary = np.empty((0,4), 'int') #picture, GFP, RFP, area

    ### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use


    smooth_filter = twodgaussian(2,5)
    smooth_filter_2 = twodgaussian(1.5,5)

    roberts_cross_v = np.array( [[ 1, 0],
                                 [ 0, -1]])

    roberts_cross_h = np.array( [[ 0, 1],
                                 [ -1, 0]])

    smoothed = ndimage.convolve( normalize(gray),smooth_filter)  #imRegs[0,2]
    vertical = ndimage.convolve( smoothed, roberts_cross_v )
    horizontal = ndimage.convolve( smoothed, roberts_cross_h )
    edge = np.sqrt( np.square(horizontal) + np.square(vertical))

    normalized = (normalize(edge)*255).astype('uint8')

    ret, thresh = cv2.threshold(normalized,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # noise removal
    kernel = np.ones((2,2),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

    # Marker labelling
    connectivity = 4  # You need to choose 4 or 8 for connectivity type  https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connected-components-with-stats-in-python
    num_markers, markers, stats, centroids = cv2.connectedComponentsWithStats(opening , connectivity , cv2.CV_16U+cv2.CC_STAT_AREA )

    # set lower limit for single cell
    threshold_single_l = 600
    # set upper limit for single cell
    sorted = np.sort([x for x in stats[:,4] if x > threshold_single_l])
    if len(sorted) > 10:
        threshold_single = np.median(sorted[0:10])*2.5
    else:
        threshold_single = 15000

    # Choose markers of area within a certain range
    chosen = np.squeeze(np.where((stats[:,4]>threshold_single_l)* (stats[:,4]<threshold_single)  # something good to read https://stackoverflow.com/questions/33747908/output-of-numpy-wherecondition-is-not-an-array-but-a-tuple-of-arrays-why
                       *(centroids[:,0]>50)*(centroids[:,0]<field_w-50)
                       *(centroids[:,1]>50)*(centroids[:,1]<field_h-50)) )

    if chosen.shape is not ():
        plt.figure(figsize = [15,15])
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1

        #plt.imshow(markers.astype(np.uint8))
        plt.imshow(gray)
        #plt.imshow(imRegs[frame,0])
        for int, c in enumerate(centroids[chosen]):
            plt.text(c[0], c[1], str(int)+'_'+str(stats[chosen[int],4]),fontsize = 20,color = 'r')
       
      
        centroids_current = centroids[chosen]
        [plt.plot(x[0],x[1],'r.') for x in centroids_current]

        plt.show()
        

    nummask = np.zeros(np.shape(markers))
    for ide, cell in enumerate(chosen+1):
        nummask[markers == cell] = ide
    nummask = nummask.astype('uint8')

    # Now, mark the region of unknown with zero

    watershed = watershed(gray,nummask)
    #img[watershed == -1] = [255,0,0]



    # calculate the sum of intensity in other channels
    record = watershed*0  
    current_frame_GFP = np.zeros(len(chosen))
    current_frame_RFP = np.zeros(len(chosen))
    for cell_ind, cell in enumerate(set([i for i in watershed.flatten()])):
        singlecellmask = np.squeeze(np.array([watershed == cell+1]).astype('int'))
        w_area = np.sum(singlecellmask) # watershed_area
        
        if (w_area < threshold_single*1.5)*(w_area>0):
            singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask).astype('int')
            current_frame_GFP[cell_ind] = singlecellGFP/w_area
            singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask).astype('int')
            current_frame_RFP[cell_ind] = singlecellRFP/w_area
            summary_current_cell = [frame,singlecellGFP,singlecellRFP, w_area]
            print(summary_current_cell)
            cellsummary = np.append(cellsummary,[summary_current_cell],axis = 0)
            record += watershed*(watershed == cell+1)
    w_chosen = chosen[list(set(record.flatten()))] #watershed chosen
    
    print(set(record.flatten()))
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,0]*record)
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(c[0], c[1], 'r.')
        plt.text(c[0], c[1], str(current_frame_GFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,1]*record)
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(c[0], c[1], 'r.')
        plt.text(c[0], c[1], str(current_frame_RFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(watershed)
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(c[0], c[1], 'r.')
        #plt.text(c[0], c[1], str(current_frame_GFP[int]))
    plt.show()
    
    plt.figure(figsize = [15,15])
    plt.imshow(imRegs[frame,1]*record)
    for int, c in enumerate(centroids[w_chosen]):
        plt.plot(c[0], c[1], 'r.')
    plt.show()

    return cellsummary

In [ ]:
extract_stats(img,frame)

In [ ]:
cellsummary = np.empty((0,4), 'int')
frames = range(len(stack))
for frame in frames:
    img = imRegs[frame,2]
    cellsummary_current = extract_stats(img,frame)
    cellsummary = np.append(cellsummary,cellsummary_current,axis=0)

In [ ]:
np.shape(cellsummary)



In [ ]:
from scipy.stats import linregress
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
Two = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
Four = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,32+33)]
Five = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(65,65+33)]
Six = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(98,98+34)]
Seven = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(132,132+35)]
Eight = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(167,167+35)]
Nine = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(202,202+37)]
strainlist = [Two,Four,Five,Six,Seven,Eight,Nine]
for strain in strainlist:
    plt.figure(figsize = [5,5])
    #x = cellsummary[strain,RFP]
    x = cellsummary[strain,area]**(3/2)
    y = cellsummary[strain,GFP]/cellsummary[strain,area]**(3/2)
    #y = cellsummary[strain,RFP]
    
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    line = slope*x+intercept
    two = plt.plot(x,y,'*')
    plt.plot( x, line)#,'r',label = "")


    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    #x=0
    #y=0
    #x = cellsummary[WHI3,RFP]
    #y = cellsummary[WHI3,GFP]/cellsummary[WHI3,area]**(3/2)
    #slope, intercept, r_value, p_value, std_err = linregress(x,y)
    #line = slope*x+intercept
    #whi3 = plt.plot(x,y,'g*')
    #plt.plot(x , line,'g',label = "WHI3")

    print(slope)
    print(r_value)
    #plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
    #plt.xlim(0,12000)
    #[print(str([x,y]) + ';') for x, y in zip(x,y)]

    plt.xlabel("RFP",size = 15)
    plt.ylabel("GFP/volume", size = 15)
    plt.legend(fontsize = 15)


In [ ]:
plt.imshow(singlecellmask)

In [ ]:
for cell in set([i for i in watershed.flatten()]):
    singlecellmask = np.squeeze(np.array([watershed == cell]).astype('int'))
    plt.imshow(singlecellmask)
    plt.show()
    w_area = np.sum(singlecellmask)

    singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask).astype('int')
    singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask).astype('int')
    summary_current_cell = [frame,singlecellGFP,singlecellRFP, stats[cell, 4]]
    print(summary_current_cell)
    cellsummary = np.append(cellsummary,[summary_current_cell],axis = 0)

In [ ]:
set((watershed == cell+1).flatten())

In [ ]:
plt.imshow(watershed*(watershed == 1))
plt.show()

In [ ]:
from scipy import ndimage
from skimage.morphology import watershed

gray = img#*(img>20)

cellsummary = np.empty((0,4), 'int') #picture, GFP, RFP, area

### Roberts  https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use


smooth_filter = twodgaussian(2,5)
smooth_filter_2 = twodgaussian(1.5,5)

roberts_cross_v = np.array( [[ 1, 0],
                             [ 0, -1]])

roberts_cross_h = np.array( [[ 0, 1],
                             [ -1, 0]])

#roberts_cross_v = np.array( [[ 1, 1],
#                             [ 0, 0]])

#roberts_cross_h = np.array( [[ 1, 0],
#                             [ 1, 0]])
#roberts_cross_1 = np.array( [[ 0, 1],
#                             [ 1, 0]])
#roberts_cross_2 = np.array( [[ 1, 0],
#                             [ 0, 1]])
#roberts_cross_3 = np.array( [[ 0, 0],
#                             [ 1, 1]])

smoothed = ndimage.convolve( normalize(gray),smooth_filter)  #imRegs[0,2]
vertical = ndimage.convolve( smoothed, roberts_cross_v )
horizontal = ndimage.convolve( smoothed, roberts_cross_h )
#ha = ndimage.convolve( smoothed, roberts_cross_1 )
#he = ndimage.convolve( smoothed, roberts_cross_2 )
#hi = ndimage.convolve( smoothed, roberts_cross_3 )
edge = np.sqrt( np.square(horizontal) + np.square(vertical))# +np.square(ha) +np.square(he)+np.square(hi))
    
normalized = (normalize(edge)*255).astype('uint8')

ret, thresh = cv2.threshold(normalized,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# noise removal
kernel = np.ones((2,2),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# Marker labelling
#ret, markers = cv2.connectedComponents(islands)
connectivity = 4  # You need to choose 4 or 8 for connectivity type  https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connected-components-with-stats-in-python
num_markers, markers, stats, centroids = cv2.connectedComponentsWithStats(opening , connectivity , cv2.CV_16U+cv2.CC_STAT_AREA )

# set lower limit for single cell
threshold_single_l = 600
# set upper limit for single cell
sorted = np.sort([x for x in stats[:,4] if x > threshold_single_l])
if len(sorted) > 10:
    threshold_single = np.median(sorted[0:10])*2.5
else:
    threshold_single = 15000

# Choose markers of area within a certain range
chosen = np.squeeze(np.where((stats[:,4]>threshold_single_l)* (stats[:,4]<threshold_single)  # something good to read https://stackoverflow.com/questions/33747908/output-of-numpy-wherecondition-is-not-an-array-but-a-tuple-of-arrays-why
                   *(centroids[:,0]>50)*(centroids[:,0]<field_w-50)
                   *(centroids[:,1]>50)*(centroids[:,1]<field_h-50)) )

if chosen.shape is not ():
    plt.figure(figsize = [15,15])
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    #plt.imshow(markers.astype(np.uint8))
    plt.imshow(gray)
    #plt.imshow(imRegs[frame,0])
    for int, c in enumerate(centroids[chosen]):
        plt.text(c[0], c[1], str(int)+'_'+str(stats[chosen[int],4]),fontsize = 20,color = 'r')

    centroids_current = centroids[chosen]
    [plt.plot(x[0],x[1],'r.') for x in centroids_current]


nummask = np.zeros(np.shape(markers))
for ide, cell in enumerate(chosen+1):
    nummask[markers == cell] = ide
nummask = nummask.astype('uint8')

# Now, mark the region of unknown with zero

watershed = watershed(gray,nummask)
#img[watershed == -1] = [255,0,0]



# calculate the sum of intensity in other channels
record = watershed*0    
for cell in set([i for i in watershed.flatten()]):
    singlecellmask = np.squeeze(np.array([watershed == cell+1]).astype('int'))
    w_area = np.sum(singlecellmask) # watershed_area
    if (w_area < threshold_single*1.5)*(w_area>0):
        singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask).astype('int')
        singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask).astype('int')
        summary_current_cell = [frame,singlecellGFP,singlecellRFP, w_area]
        print(summary_current_cell)
        cellsummary = np.append(cellsummary,[summary_current_cell],axis = 0)
        record += watershed*(watershed == cell+1)
plt.imshow(record)
plt.show()



print('opening:')
plt.figure(figsize=[15,15])
plt.imshow(opening)
for int, c in enumerate(centroids[chosen]):
    plt.text(c[0], c[1], str(int)+'_'+str(stats[chosen[int],4]),fontsize = 20,color = 'r')
plt.colorbar()
plt.show()

print('makers:')
plt.figure(figsize=[15,15])
plt.imshow(markers)
plt.colorbar()
plt.show()

plt.figure(figsize = [15,15])
plt.imshow(nummask)
plt.colorbar()
plt.show()

plt.figure(figsize=[15,15])
plt.imshow(watershed)
plt.colorbar()
plt.show()


In [ ]:
np.max(watershed)

In [ ]:
plt.imshow(watershed)

In [ ]:
plt.figure(figsize=[15,15])
plt.imshow(imRegs[frame,0]/watershed)

In [ ]:
gray = img

ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# noise removal
kernel = np.ones((2,2),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)


# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

print('gray')
plt.figure(figsize=[15,15])
plt.imshow(gray)
plt.colorbar()
plt.show()

print('threshold')
plt.figure(figsize=[15,15])
plt.imshow(thresh)
plt.colorbar()
plt.show()

print('opening:')
plt.figure(figsize=[15,15])
plt.imshow(50*opening)
plt.colorbar()
plt.show()

print('sure_bg:')
plt.figure(figsize=[15,15])
plt.imshow(sure_bg)
plt.colorbar()
plt.show()

print('sure_fg:')
plt.figure(figsize=[15,15])
plt.imshow(sure_fg)
plt.colorbar()
plt.show()

In [ ]:

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

In [ ]:
#frames = range(5)
frames = range(len(stack))
ax = fig.add_subplot(111)
cellsummary = np.empty((0,4), int) #picture, GFP, RFP, area


for ind, frame in enumerate(frames):
    fig = plt.figure(figsize = [15,15]) 
    # Copy the thresholded image. https://docs.opencv.org/3.1.0/d3/db4/tutorial_py_watershed.html
    im_th = np.array(normalize(stack[frame])*255>5).astype(int)
    im_floodfill = im_th
    plt.imshow(im_th)
    plt.show()
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (2,2), 255); # seed it from somewhere other than the edge...sometimes the edges have wired values

    # Invert floodfilled image
    islands = cv2.bitwise_not(im_floodfill).astype(np.uint8)
    
    # Marker labelling
    #ret, markers = cv2.connectedComponents(islands)

    connectivity = 4  # You need to choose 4 or 8 for connectivity type  https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connected-components-with-stats-in-python
    num_markers, markers, stats, centroids = cv2.connectedComponentsWithStats(islands , connectivity , cv2.CV_16U+cv2.CC_STAT_AREA )
    
    # set upper limit for single cell
    sorted = np.sort([x for x in stats[:,4] if x > 1000])
    if len(sorted) > 5:
        threshold_single = np.median(sorted[0:5])*1.5
    else:
        threshold_single = 15000
        
    # Choose markers of area within a certain range
    chosen = np.squeeze(np.where((stats[:,4]>1000)* (stats[:,4]<threshold_single)  # something good to read https://stackoverflow.com/questions/33747908/output-of-numpy-wherecondition-is-not-an-array-but-a-tuple-of-arrays-why
                       *(centroids[:,0]>50)*(centroids[:,0]<field_w-50)
                       *(centroids[:,1]>50)*(centroids[:,1]<field_h-50)) )
    print(chosen)
    if chosen.shape is not ():
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1

        plt.imshow(markers.astype(np.uint8))
        #plt.imshow(imRegs[frame,0])
        #for c in centroids[chosen]:
        #    plt.text(c[0], c[1], str(c))

        centroids_current = centroids[chosen]
        [plt.plot(x[0],x[1],'r.') for x in centroids_current]
        #print(frame,num_markers,stats)

        # show the image
        plt.show()
        #time.sleep(3)
        #display.clear_output(wait=True)

        # calculate the sum of intensity in other channels
    
        for cell in chosen:
            singlecellmask = np.squeeze(np.array([markers == cell+1]).astype(int))
            singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask).astype(int)
            singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask).astype(int)
            summary_current_cell = [frame,singlecellGFP,singlecellRFP, stats[cell, 4]]
            #print(summary_current_cell)
            cellsummary = np.append(cellsummary,[summary_current_cell],axis = 0)
    print(frame)
    #plt.hist(stats[chosen,3])
    #plt.xlim(0,255)
    #plt.show()
    #display.clear_output(wait=True)

#outfile = open('090119.npy','wb')
#np.save(outfile,cellsummary)
        

In [ ]:
centroids.astype(int)

In [ ]:
field_w

In [ ]:
plt.hist(stats[chosen,3])

In [ ]:
np.shape(cellsummary)

In [ ]:
chosen.shape == ()

In [ ]:
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3

CLN3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
HTA2 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,63)]
WHI3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(63,96)]
plt.figure(figsize = [15,15])
plt.plot(cellsummary[CLN3,area],cellsummary[CLN3,GFP],'g*')
plt.plot(cellsummary[WHI3,area],cellsummary[WHI3,GFP],'r*')
plt.plot(cellsummary[HTA2,area],cellsummary[HTA2,GFP],'y*')
plt.xlim(0,20000)
#plt.ylim(0,20)

In [ ]:
from scipy.stats import linregress
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
CLN3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
HTA2 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,63)]
WHI3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(63,96)]
plt.figure(figsize = [5,5])
x = cellsummary[CLN3,RFP]
y = cellsummary[CLN3,GFP]/cellsummary[CLN3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
cln3 = plt.plot(x,y,'r*')
plt.plot( x, line,'r',label = "CLN3")


#[print(str([x,y]) + ';') for x, y in zip(x,y)]

x=0
y=0
x = cellsummary[WHI3,RFP]
y = cellsummary[WHI3,GFP]/cellsummary[WHI3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
whi3 = plt.plot(x,y,'g*')
plt.plot(x , line,'g',label = "WHI3")

print(slope)
print(r_value)
#plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
plt.xlim(0,12000)
#[print(str([x,y]) + ';') for x, y in zip(x,y)]

plt.xlabel("RFP",size = 15)
plt.ylabel("GFP/volume", size = 15)
plt.legend(fontsize = 15)





In [ ]:
[[i,j] for i, j in zip(x,y)][:]

In [ ]:
from scipy.stats import linregress
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
CLN3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
HTA2 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,63)]
WHI3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(63,96)]
plt.figure(figsize = [5,5])
x = cellsummary[CLN3,area]**(3/2)
y = cellsummary[CLN3,GFP]/cellsummary[CLN3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
cln3 = plt.plot(x,y,'r*')
plt.plot( x, line,'r',label = "CLN3")


#[print(str([x,y]) + ';') for x, y in zip(x,y)]

x=0
y=0
x = cellsummary[WHI3,area]**(3/2)
y = cellsummary[WHI3,GFP]/cellsummary[WHI3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
whi3 = plt.plot(x,y,'g*')
plt.plot(x , line,'g',label = "WHI3")

print(slope)
print(r_value)
#plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
#plt.xlim(0,12000)
#[print(str([x,y]) + ';') for x, y in zip(x,y)]

plt.xlabel("RFP",size = 15)
plt.ylabel("GFP/volume", size = 15)
plt.legend(fontsize = 15)





In [ ]:
from scipy.stats import linregress
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
CLN3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
HTA2 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,63)]
WHI3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(63,96)]
plt.figure(figsize = [5,5])
x = cellsummary[CLN3,RFP]
y = cellsummary[CLN3,GFP]#/cellsummary[CLN3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
cln3 = plt.plot(x,y,'r*')
plt.plot( x, line,'r',label = "CLN3")


#[print(str([x,y]) + ';') for x, y in zip(x,y)]

x=0
y=0
x = cellsummary[WHI3,RFP]
y = cellsummary[WHI3,GFP]#/cellsummary[WHI3,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
whi3 = plt.plot(x,y,'g*')
plt.plot(x , line,'g',label = "WHI3")

print(slope)
print(r_value)
#plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
plt.xlim(0,12000)
#[print(str([x,y]) + ';') for x, y in zip(x,y)]



x=0
y=0
x = cellsummary[HTA2,area]
y = cellsummary[HTA2,RFP]#/cellsummary[HTA2,area]**(3/2)
slope, intercept, r_value, p_value, std_err = linregress(x,y)
line = slope*x+intercept
Hta2 = plt.plot(x,y,'y*')
plt.plot(x , line,'y',label = "HTA2")

print(slope)
print(r_value)
#plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
plt.xlim(0,12000)
#[print(str([x,y]) + ';') for x, y in zip(x,y)]


In [ ]:
np.sort(cellsummary[WHI3,area])

In [ ]:
#picture, GFP, RFP, area
GFP = 1
RFP = 2
area = 3
CLN3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32)]
HTA2 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(32,63)]
WHI3 = [i for i, ind in enumerate(cellsummary[:,0]) if ind in range(63,96)]
plt.figure(figsize = [15,15])
plt.plot(cellsummary[CLN3,RFP],cellsummary[CLN3,GFP]/cellsummary[CLN3,RFP],color = 1)
#plt.plot(cellsummary[WHI3,RFP],cellsummary[WHI3,GFP]/cellsummary[WHI3,RFP],'r*')
#plt.plot(cellsummary[HTA2,3],cellsummary[HTA2,2],'y*')
plt.xlim(0,20000)

In [ ]:
frame

In [ ]:
a = np.append(np.empty((0,4), int),[framesummary],axis = 0)
print(a)

In [ ]:
[i for i in stats[chosen]]


In [ ]:
np.squeeze(imRegs[9,0]*[islands == 1])

In [ ]:
chosen

In [ ]:
np.max([i for i in np.array([markers == 62]).astype(int)])

In [ ]:
for cell in chosen:
    singlecellmask = np.squeeze(np.array([markers == cell+1]).astype(int))
    singlecellGFP = np.sum(imRegs[frame,0] * singlecellmask)
    singlecellRFP = np.sum(imRegs[frame,1] * singlecellmask)
    print([singlecellGFP,singlecellRFP])

In [ ]:
sum([i for i in sum(imRegs[9,1] * np.squeeze(np.array([markers == chosen[1]+1]).astype(int)))])

In [ ]:
plt.figure(figsize = [15,15])
plt.imshow(imRegs[9,1] * np.squeeze(np.array([markers == chosen[1]+1]).astype(int)))
plt.show()